<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Simple Decision Prompts

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using GenAI prompt engineering  
* Creating custom AI personas with a system decision prompt  
* Generating decision outputs using easier to consume formats (Markdown)     

Prompts are the fundamental building block to getting the proper responses from AI models. This module demonstrates how to use common prompt engineering techniques while using Semantic Kernel. If you've ever used ChatGPT or Microsoft Copilot, you're already familiar with prompting. Given a prompt instruction, an LLM will attempt to predict the most likely response. With Semantic Kernel, you can craft applications, services & APIs that execute prompts.  

For more information on using prompts with Semantic Kernel, visit: https://learn.microsoft.com/en-us/semantic-kernel/prompts/your-first-prompt  

The process of carefully crafting questions or instructions for AI is called **Prompt Engineering**. Prompt Engineering involves designing and refining input prompts—text or questions—so that the AI produces responses that are more relevant, useful, or accurate. The goal of prompt engineering is to "tune" the prompt in a way that maximizes the quality and clarity of the AI's response, often using specific wording, context, or examples within the prompt itself.

Decision Intelligence and Prompt Engineering techniques can be combined to created "Generative AI Decision Intelligence"! This can leverage GenAI models to reason over complex tasks, gather intelligence for decisions, recommend decisions, communicate decisions etc. 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [7]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.5"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.5"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connection info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.5 Microsoft.Extensions.Configuration.Json, 9.0.5

In [2]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.54"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Installed Packages Microsoft.SemanticKernel, 1.54.0

Using Azure OpenAI Service


### Step 2 - Execute a Decision Prompt 

Many LLMs and even SLMs have been trained on knowledge that includes decision frameworks & processes. This makes LLMs great assistants to:
* Provide proper Decision Frames
* Gather Intelligence (information) in order to make a decision
* Recommend Decision Frameworks to make a high-quality decision
* Provide feedback to evaluate a Decision

Once the Semantic Kernel instance is built, it is ready to intake prompt instructions. In the prompt below, the Semantic Kernel is instructed to provide examples of decision frameworks "trained into" the knowledge of the configured AI model.  

In [ ]:
// A Decision Intelligence prompt to help with describing decision-making frameworks
var simpleDecisionPrompt = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios.
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt);
var response = simplePromptResponse.GetValue<string>(); 

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

Here are five decision-making frameworks that can enhance decision quality, along with descriptions of how they support better analysis and reasoning:

---

### 1. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**
- **Description**: This framework systematically evaluates internal strengths and weaknesses and external opportunities and threats. It provides a structured lens through which to assess a situation or decision.
- **How it supports better analysis**: SWOT helps identify positive and negative factors in both internal and external environments, promoting a balanced approach to decision-making. This ensures that potential risks and advantages are thoughtfully considered before taking action.

---

### 2. **Pareto Principle (80/20 Rule)**
- **Description**: This principle focuses on identifying the critical factors that will yield the most significant impact, emphasizing prioritization of the "vital few" over the "trivial many."
- **How it supports better analysis**: By helping decision-makers concentrate on the most influential inputs or actions, the Pareto Principle prevents wasted effort on less impactful details, allowing for more focused, effective choices in resource allocation and problem-solving.

---

### 3. **Decision Matrix**
- **Description**: This tool involves creating a grid that evaluates multiple options based on a set of weighted criteria. Each option is scored for each criterion to identify the best choice.
- **How it supports better analysis**: A decision matrix ensures that decisions are made based on objective comparisons rather than subjective preferences. By quantifying factors, it reduces bias and aligns choices with defined priorities and goals.

---

### 4. **Cost-Benefit Analysis**
- **Description**: This framework involves comparing the costs of an action or decision with its potential benefits, quantifying value wherever possible.
- **How it supports better analysis**: Cost-benefit analysis ensures that the economic and resource implications of a decision are fully understood. It helps decision-makers weigh trade-offs, ensuring optimal allocation of resources based on efficiency and outcomes.

---

### 5. **Six Thinking Hats**
- **Description**: This framework categorizes thinking into six perspectives: facts (White), emotions (Red), caution (Black), optimism (Yellow), creativity (Green), and process oversight (Blue). By role-playing different "hats," participants explore a problem from multiple angles.
- **How it supports better analysis**: Six Thinking Hats fosters a balanced and multidimensional approach to decision-making. It ensures diverse viewpoints are considered, reducing blind spots and improving collaborative reasoning by integrating logic, emotions, creativity, and risk assessment.

---

These frameworks help improve decision-making by providing structured methods to assess options, reduce bias, and anticipate consequences in complex scenarios. Using them ensures that decisions are grounded in logic, evidence, and careful prioritization.

### Step 3 - Execute a Decision Prompt with Streaming

Semantic Kernel supports prompt response streaming when invoking the prompt. This allows responses to be streamed to the client as soon as they are made available by the LLM and service. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

> 📝 Note: An average human can read between 25-40 Tokens / second. Therefore, while streaming certainly helps with providing AI output to the user, it begins to lose its effectiveness at large token velocity.  

In [5]:
// Decision Prompt using Streaming output chunks 
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt))
{
   Console.Write(streamChunk);
}

### Step 4 - Execute a Decision Prompt with Improved Output Formatting  

Generative AI models have an inherent ability to not only provide decision reasoning analysis, but also format the output in a desired format. This could be as simple as instructing the Generative AI model to format the decision as a single sentence, paragraphs or lists. However more sophisticated output generations can be instructed. For example, the GenAI model can output Markdown or even extract information and fill in a desired schema (JSON). Specifically for Decision Intelligence, you can ask the GenAI models to apply decision communication frameworks to the generation as well. 

Execute the simple decision prompt below with Markdown formatting output. This table can now be rendered in a Markdown document for easy human comprehension. Markdown tables and other formats can be used on web sites, document, programming code etc. Even Generative AI models understand Markdown format, which can not only be used for output but inside input prompts.  

In [4]:
// A new decision prompt to help with describing decision-making frameworks using Markdown format
var simpleDecisionPromptWithMarkdownFormat = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios.

Format the response using only a Markdown table. Do not enclose the table in triple backticks.
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPromptWithMarkdownFormat);
var response = simplePromptResponse.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

| Framework               | Description                                                                                                 |
|--------------------------|-------------------------------------------------------------------------------------------------------------|
| **SWOT Analysis**        | Examines Strengths, Weaknesses, Opportunities, and Threats to help identify internal capabilities and external factors. Enhances decision quality by providing a balanced view of potential risks and gains. |
| **Cost-Benefit Analysis**| Compares the costs and benefits of different options to determine the most financially or operationally viable choice. Encourages logical thinking and resource optimization.           |
| **Decision Matrix**      | Utilizes weighted criteria to compare multiple options quantitatively, enabling structured evaluation. Supports logical, justified, and transparent decisions.                    |
| **The 5 Whys**           | Iteratively asks "Why?" to drill down into the root cause of a problem. Aids in thorough problem-solving and avoids superficial resolutions.                                      |
| **Pareto Analysis**      | Focuses on identifying the 20% of causes that lead to 80% of the effects (80/20 rule). Prioritizes high-impact areas to maximize efficiency and outcomes.                         |

### Step 5 - Execute a Decision Prompt with a Custom Prompt Execution Configuration

Semantic Kernel supports the configuration of prompt execution. The typical OpenAI and Azure OpenAI settings are exposed as configuration options that provide a different prompt experience. Try changing the MaxTokens, Temperature or other settings.

> 📝 Note: The supported settings are dependent on the API plus the specific model version. For example, a model paired with an older API may not expose all the configuration settings available. Conversely, a new preview model may not have all the settings available until it becomes generally available.  

In [5]:
// Create a new OpenAI Prompt Execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

// Create a new KernelArguments object with the OpenAI prompt execution settings
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

var promptResponseWithOpenAI = await semanticKernel.InvokePromptAsync(simpleDecisionPromptWithMarkdownFormat, kernelArguments);
var response = promptResponseWithOpenAI.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

| Decision-Making Framework | Description |
|----------------------------|-------------|
| **SWOT Analysis**          | Focuses on identifying Strengths, Weaknesses, Opportunities, and Threats related to a decision. It helps in understanding internal and external factors, enabling balanced and strategic choices. |
| **Cost-Benefit Analysis**  | Compares the costs and benefits of different options quantitatively. This framework ensures decisions are economically viable and prioritize options with the highest net benefit. |
| **Decision Matrix**        | Uses a grid to evaluate multiple alternatives against various criteria. It supports objective comparison and ranking of options based on weighted factors. |
| **Pareto Analysis**        | Focuses on identifying the most impactful factors (80/20 rule). It helps prioritize efforts on areas that yield the greatest benefits, improving efficiency in decision-making. |
| **Six Thinking Hats**      | Encourages analyzing decisions from six perspectives: logic, emotion, creativity, optimism, caution, and process. This framework fosters comprehensive reasoning and minimizes bias. |

### Step 6 - Execute a Decision Prompt with a System Prompt (Custom AI Persona)

When building Decision Intelligence prompts, all the typical best practices of prompt engineering apply when using Semantic Kernel.  Such as: 
* Make the prompt more specific (i.e. decision intelligence)
* Add structure to the output with formatting
* Provide examples with few-shot prompting
* Instruct the AI what to do to avoid doing something else
* Provide context (additional information) to the AI
* Using Roles in Chat Completion API prompts
* Give your AI words of encouragement  

A key best practice is to provide a common behavior across all the LLM interactions in a system prompt. This system prompt is passed in on every single call the LLM with Semantic Kernel. By passing the same (or similar) system prompt with every prompt gives your Generative AI system a common persona. This common persona is the foundational building block of building AI agents; where the desired behavior is to have each agent have a unique persona/behavior every time you interact with that agent.

Execute the cell below with a dynamic prompt template. Notice the different behavior of the output for decision frameworks. Based on the new system prompt instructions, the decision framework responses are much more robust with decision intelligence information.

In [6]:
// Create a System Prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a Decision Intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive decision advice.
""";
var simpleDecisionPrompt = """
Recommend the top 5 decision frameworks that can be used for daily situations to make various decisions.
These frameworks should be very easy to understand and apply to various scenarios.
""";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: 
{systemPromptForDecisions}

Request from the user: 
{simpleDecisionPrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

var promptResponseWithTemplate = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt, kernelArguments);
var response = promptResponseWithTemplate.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

Here are five simple and versatile decision-making frameworks that can be applied to a wide range of daily situations. Each is easy to understand and implement, making them ideal for quick, practical use:

---

### 1. **The Eisenhower Matrix (Urgent-Important Matrix)**  
**Purpose:** Prioritize tasks and decisions based on urgency and importance.  
**How It Works:**  
- Divide tasks or decisions into four quadrants:  
  1. **Urgent and Important**: Do it immediately.  
  2. **Important but Not Urgent**: Schedule it for later.  
  3. **Urgent but Not Important**: Delegate it if possible.  
  4. **Neither Urgent nor Important**: Eliminate or ignore it.  
**Example Use Case:** Deciding how to allocate your time during a busy day.  

---

### 2. **The 80/20 Rule (Pareto Principle)**  
**Purpose:** Focus on the 20% of actions that yield 80% of the results.  
**How It Works:**  
- Identify the key tasks or decisions that will have the greatest impact.  
- Prioritize those and minimize time spent on less impactful activities.  
**Example Use Case:** Choosing which work tasks to focus on to maximize productivity.  

---

### 3. **The 5 Whys Technique**  
**Purpose:** Get to the root cause of a problem or decision.  
**How It Works:**  
- Ask "Why?" five times (or as many times as needed) to uncover the underlying cause or motivation.  
- Use the insights to make an informed decision.  
**Example Use Case:** Deciding how to address recurring issues, like why you're always late to meetings.  

---

### 4. **The OODA Loop (Observe, Orient, Decide, Act)**  
**Purpose:** Make decisions quickly and adapt to changing circumstances.  
**How It Works:**  
- **Observe:** Gather information about the situation.  
- **Orient:** Analyze the information and understand the context.  
- **Decide:** Choose the best course of action.  
- **Act:** Implement the decision and monitor the outcome.  
**Example Use Case:** Deciding how to respond to unexpected challenges, like a last-minute schedule change.  

---

### 5. **The 10/10/10 Rule**  
**Purpose:** Evaluate the long-term impact of a decision.  
**How It Works:**  
- Ask yourself:  
  - How will I feel about this decision in **10 minutes**?  
  - How will I feel about it in **10 months**?  
  - How will I feel about it in **10 years**?  
- Use the answers to weigh short-term vs. long-term consequences.  
**Example Use Case:** Deciding whether to make a quick emotional purchase or save money for a future goal.  

---

### Summary Table for Quick Reference:

| Framework         | Key Focus              | Best For                                  |
|--------------------|------------------------|-------------------------------------------|
| Eisenhower Matrix  | Urgency vs. Importance| Time management and prioritization        |
| 80/20 Rule         | High-impact actions   | Maximizing efficiency and results         |
| 5 Whys             | Root cause analysis   | Problem-solving and understanding motives |
| OODA Loop          | Quick adaptability    | Responding to dynamic or changing situations |
| 10/10/10 Rule      | Long-term perspective | Balancing short-term and long-term impacts|

---

### How to Choose the Right Framework:
- **For prioritizing tasks:** Use the Eisenhower Matrix or 80/20 Rule.  
- **For solving problems:** Use the 5 Whys.  
- **For dynamic situations:** Use the OODA Loop.  
- **For weighing long-term impacts:** Use the 10/10/10 Rule.  

By keeping these frameworks in your decision-making toolkit, you can approach daily situations with clarity and confidence. Let me know if you'd like help applying any of these to a specific scenario!

### Step 7 - Execute a Decision Scenario with a System Prompt (Custom AI Persona)

In this step a decision scenario will be introduced requiring analysis and a recommendation performed by Artificial Intelligence. The full Decision Intelligence framework will not be used, rather a simple request for Artificial Intelligence to recommend a path forward (recommendation) for the human user to ultimately make the final decision.  

**Decision Scenario:** Your high school daughter Alex is deciding whether to enroll directly in a four-year university or start at a community college to earn an associate degree first. These are Alex's main decision factors: financial, career uncertainty, academic consistency and future impact. In addition, you have all the decision factor detailed data available to pass to the GenAI model prompt. You are looking for an impartial (non-family) recommendation. Can Artificial Intelligence be that impartial judge to help Alex decide? 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-SimpleDecision-College.png">

In [10]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive decision advice.
""";
// Provide a description of the decision scenario and the desired output 
// Provide detailed Decision Scenario considerations and information about Alex (the decision-maker) 
var scenarioDecisionPrompt = """
Imagine you are advising a daughter named Alex who is deciding whether to enroll directly in a well-regarded four-year 
university or start at a community college to earn an associate degree first. 

Make a single recommendation based on the following decision factor details. 
Output the recommendation in a Markdown table format. 

Financial Considerations:
Alex's four-year university will cost significantly more in tuition, housing, and related expenses 
(estimated $50,000-$60,000 per year). A two-year associate program at a local community college could 
save substantial money (estimated $3,000-$5,000 per year), but Alex may have to transfer to a 
four-year institution later to complete a bachelor's degree. The family can afford the four-year university, 
with some loans, but the cost is only a medium concern. 

Career and Major Uncertainty:
Alex is not entirely sure what she wants to major in. She is torn between business, psychology, and 
possibly something in the arts. She worries that if she starts at the four-year university, 
she might switch majors and incur extra time and cost. On the other hand, 
community college might give her space to explore options, 
but transferring could mean adjusting to a new campus and curriculum midway through.

Academic Consistency and Networking:
Going straight to the four-year university would allow Alex to build early relationships with professors, 
join campus groups, and potentially secure internships or research opportunities. Starting at community college might 
delay those opportunities, but it could also let her explore different fields at a lower cost. 
She might miss out on the “full campus” experience early on, but transferring later means she could 
still build connections, just on a different timeline. 

Future Impact: 
Alex is unsure of the short-term future impact of her decision that might be hard to remediate. 
Alex wants a solid professional network and relevant experience when she graduates. 
Alex is not overly concerned about the social aspect of college, 
but feels she can build a quality network in a four-year university setting sooner. 
She is also concerned about taking on significant student loan debt. The decision affects not only her immediate academic path but 
also her long-term financial stability and career prospects. 
""";

// How the prompt looks like to the LLM
var scenarioDecisionPromptTemplate = $"""
System Prompt: 
{systemPromptForDecisions}

Request from the user: 
{scenarioDecisionPrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 1000, // Increase the max tokens to allow for a more detailed response
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

var promptResponseScenario = await semanticKernel.InvokePromptAsync(scenarioDecisionPromptTemplate, kernelArguments);
var response = promptResponseScenario.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

| **Decision Factor**                | **Four-Year University (Direct Entry)**                                                                 | **Community College (Associate First, Then Transfer)**                                         | **Analysis & Recommendation**                                                                                                                                                                                                                                                                                                                                                     |
|------------------------------------|--------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Financial Considerations**       | High cost ($50k-$60k/year); family can afford with some loans; medium concern.                         | Low cost ($3k-$5k/year); significant savings; may require loans for later transfer.            | Community college offers substantial savings, reducing future debt and financial pressure. While the family can afford the university, minimizing debt is prudent, especially given Alex’s uncertainty about her major and career path.                                                                                                                                            |
| **Career & Major Uncertainty**     | Early commitment to a major; switching majors could increase time/cost; less flexibility to explore.   | More flexibility to explore majors at lower cost; easier to change direction early on.         | Community college provides a low-risk environment for academic exploration, which is ideal given Alex’s uncertainty. This reduces the risk of costly mistakes if she changes majors.                                                                                                                |
| **Academic Consistency & Networking** | Immediate access to university resources, professors, and networking; early campus integration.        | Delayed access to university-level networking; possible adjustment challenges when transferring. | While direct entry offers earlier networking, community college students can still build strong networks after transferring. Many universities have robust transfer programs to integrate new students. The initial delay is offset by the financial and academic flexibility benefits.                                                         |
| **Future Impact**                  | Potential for stronger early professional network; higher debt load; less flexibility if uncertain.    | Lower debt; more flexibility; may need to work harder to build network after transferring.      | Lower debt improves long-term financial stability. While networking starts later, Alex can still achieve her goals with intentional effort after transferring. The flexibility and reduced financial risk of starting at community college outweigh the benefits of immediate university entry, especially given Alex’s current uncertainty about her major and career path.                       |

### **Recommendation**

**Based on the factors provided, Alex should start at a community college to earn an associate degree, then transfer to a four-year university to complete her bachelor’s degree.**

**Rationale:**  
- This path offers significant financial savings and minimizes student loan debt, which is important for long-term stability.
- Community college allows Alex to explore different majors at a lower cost, reducing the risk of expensive mistakes if she changes direction.
- While there is a delay in building a university network, transfer students can still access campus resources, internships, and professional connections after transferring.
- The flexibility and reduced financial risk of starting at community college outweigh the benefits of immediate university entry, especially given Alex’s current uncertainty about her major and career path.